In [ ]:
import numpy as np
import itertools
import re
from joblib import Parallel, delayed
############ FUNCTIONS ################
def epsilon_boundary(allvalues, eps):
    return [e[0] for e in allvalues if(e[1]<=eps)]
#################

def Merge_Test_Hom0(Hom_Basis, Hom_Degree, Index, Complex1, Complex2):
    try:
        diff = Hom_Basis[Hom_Degree, Index[0]].to_cycle()-Hom_Basis[Hom_Degree, Index[1]].to_cycle()
        if diff!=0:
            mylist = list(diff.to_vector())   
            mydim = Complex1.differential(Hom_Degree+1).dimensions()
            mydim1 = Complex2.differential(Hom_Degree+1).dimensions()
            for i in range(mydim1[0]-mydim[0]):
                mylist.append(0)
            mytuple= tuple(mylist)
            myoriginal = Complex2.differential(Hom_Degree+1)
            mylist = list(Complex2.differential(Hom_Degree+1).transpose())
            mylist.append(mytuple)
            mylist = [list(tp) for tp in mylist] 
            if rank(myoriginal)==rank(matrix(mylist)):
                return(Hom_Basis[Hom_Degree,Index[0]].to_cycle(),Hom_Basis[Hom_Degree,Index[1]].to_cycle())
    except TypeError:
        pass

###############

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

##############
def create_matrix_oneper(indexes, epsilon2, Matrix_Size, fill):
    rows = [ int(re.findall(r"\d+", str(index[0]) )[0]) for index in indexes]
    columns = [ int(re.findall(r"\d+", str(index[1]) )[0]) for index in indexes]
    A = fill*np.ones((Matrix_Size, Matrix_Size))
    for i in range(len(rows)):
        A[rows[i]][columns[i]] = epsilon2
        A[columns[i]][rows[i]] = epsilon2
    A = A - fill*np.eye(Matrix_Size)
    return(A)
##############
def create_matrix_all(Data, epsilon1, epsilon2, Matrix_Size, Hom_Degree, fill ):

    X1=SimplicialComplex(epsilon_boundary(Data,epsilon1), is_immutable=True)
    X2=SimplicialComplex(epsilon_boundary(Data,epsilon2), is_immutable=True)
    CC1 = X1.n_chains(2, GF(2)).chain_complex()
    CC2 = X2.n_chains(2, GF(2)).chain_complex()
    HX1_basis= X1.homology_with_basis(GF(2)).basis(Hom_Degree)
    indexes_class = itertools.combinations(range(CC1.betti(Hom_Degree)),2)     
    indexes = Parallel(n_jobs=-1, prefer="threads", verbose = 2)(delayed(Merge_Test_Hom0)(
        Hom_Basis = HX1_basis,
        Hom_Degree = Hom_Degree,
        Index = i,
        Complex1 = CC1,
        Complex2 = CC2,
    ) for i in indexes_class)
    indexes = remove_values_from_list(indexes, None)
    
    return(create_matrix_oneper(indexes, epsilon2, Matrix_Size, fill=fill))

def distancematrix(allcomplexes, dimmatrix, n, eps1, eps2, step, fill, filename):
    epsilon1 = np.arange(eps1, eps2+step, step)

    for eps1, eps2 in zip(epsilon1[:-1],epsilon1[1:]):
        matrix = create_matrix_all(Data = allcomplexes,
                              epsilon1=eps1,
                              epsilon2=eps2,
                              Matrix_Size=dimmatrix,
                              Hom_Degree=n,
                                fill = fill)
        temp = filename +str(eps1) + "_" + str(eps2)
        np.save(temp, matrix)
        del matrix

####################################################################################################
dataname = "iris"
import json
with open("Complexes_Data/"+dataname+"_complexes.txt", "r") as file:
    allcomplexes=json.loads(file.read())


filename = "./Matrix_Data/"+dataname+"_result/"+dataname+"_distances"
distancematrix(allcomplexes =  allcomplexes,
                   dimmatrix=150,
                   n=0,
                   eps1=0.0, eps2=0.8, step=0.05.0, fill = 10 , filename = filename)